In [1]:
from mcgocr import Craft, GoData, CandidateFinder, SolidExtractor, CandidateFinder, Index, LabelMarker
from mcgocr import bulk_measurements, recover, evaluate, Evidence, Statement, Entity, Pattern, Constraint
from experiment import Report, MetaReport, Corpus, Sentence, Candidate

In [2]:
import pickle
with open('data/for_clear_mind_report.pickle', 'rb') as f:
    report = pickle.load(f)
    
craft = Craft('data')
corpus = craft.get_corpus()
goldstandard = craft.get_goldstandard()
godata = GoData.load('data/go/go_20071128.pickle')

# Load
with open('data/auxiliary_term_indeces.pickle', 'rb') as f:
    term_indeces = pickle.load(f)

with open('data/ten_fold_corpora.pickle', 'rb') as f:
    corpora = pickle.load(f)

Reading the corpus...
Reading the goldstandard of GO...


In [7]:
auxiliary_extractor = SolidExtractor(sum(term_indeces.values(), Index()))

In [8]:
%%time
finder = CandidateFinder(godata, auxiliary_extractor)

CPU times: user 58.1 s, sys: 6.43 s, total: 1min 4s
Wall time: 1min 30s


In [3]:
import random
random.sample(report.fn, 10)

[('17020410', 'GO:0033592', 8487, 8497, 'hybridised'),
 ('16279840', 'GO:0032502', 1450, 1458, 'assemble'),
 ('17194222', 'GO:0000739', 61859, 61869, 'hybridized'),
 ('16670015', 'GO:0009058', 35233, 35242, 'producing'),
 ('11897010', 'GO:0000739', 16072, 16081, 'annealing'),
 ('15005800', 'GO:0048286', 35428, 35450, 'development of alveoli'),
 ('17194222', 'GO:0000739', 50613, 50626, 'hybridization'),
 ('15207008', 'GO:0004364', 49751, 49754, 'GST'),
 ('17194222', 'GO:0009058', 46459, 46469, 'Generation'),
 ('16539743',
  'GO:0045941',
  44024422,
  44174435,
  'upregulation of ... transcription')]

In [4]:
def diagnose(item):
    pmid, wanted_goid, start, end, text = item
    report = None
    sentence = Sentence(text, 0, 'diagnosing')
    grounds = finder.extractor.to_grounds(sentence)
    candidates = finder.recognizer.generate(grounds)
    if len(grounds) == 0:
        report = 0
    elif len(candidates) == 0:
        print(grounds)
        report = 1
    elif all([wanted_goid not in c.statement.statid for c in candidates]):
        print(grounds)
        print(godata[wanted_goid].statements)
        report = 2
    return report

In [5]:
diagnose(('17244351', 'GO:0065007', 12413, 12420, 'control'))

NameError: name 'finder' is not defined

In [29]:
goid = 'GO:0065007'
text = 'control'
sentence = Sentence(text, 0, 'diagnosing')
for statement in godata[goid].statements:
    terms = statement.terms()
    if len(terms) == 1:
        term = terms[0]
        if text != term.lemma:
            print(text, term)
            print(finder.recognizer.stat_index[term])

control Pattern(lemma='regulate', ref='annotator')
{Statement(statid='GO:0050795%000', evidences=[Evidence(term=Pattern(lemma='regulate', ref='annotator'), text='regulate', start=0, end=10), Evidence(term=Pattern(lemma='behaviorresponse', ref='annotator'), text='behaviorresponse', start=14, end=22)]), Statement(statid='GO:0052031%000', evidences=[Evidence(term=Pattern(lemma='regulate', ref='annotator'), text='regulate', start=0, end=10), Evidence(term=Constraint(lemma='symbiont of host defense response', ref='GO:0044416'), text='symbiont of host defense response', start=14, end=47)]), Statement(statid='GO:0052301%000', evidences=[Evidence(term=Pattern(lemma='regulate', ref='annotator'), text='regulate', start=0, end=10), Evidence(term=Constraint(lemma='organism of defense-related calcium ion flux in other organism during symbiotic interaction', ref='GO:0052301'), text='organism of defense-related calcium ion flux in other organism during symbiotic interaction', start=14, end=105)]), St

In [45]:
import progressbar
from collections import defaultdict

bar = progressbar.ProgressBar()
pmid2name = dict()
ten_corpora = corpus.divide(10)
for corp in ten_corpora:
    pmid2name.update({pmid: corp.title for pmid in corp.doc_set()})
    
term_indeces = defaultdict(Index)
for pmid, goid, start, end, text in bar(goldstandard):
    if '...' in text:
        continue
    name = pmid2name[pmid]
    for statement in godata[goid].statements:
        terms = statement.terms()
        if len(terms) == 1:
            term = terms[0]
            #if text != term.lemma:
            term_indeces[name][text].add(term)


100% (29447 of 29447) |###################| Elapsed Time: 0:00:01 Time: 0:00:01


In [46]:
termindex = sum(term_indeces.values(), Index())

In [47]:
termindex['form']

{Pattern(lemma='develop', ref='annotator')}

In [48]:
[s for s in godata['GO:0032502'].statements if len(s.terms())==1]

[Statement(statid='GO:0032502%001', evidences=[Evidence(term=Pattern(lemma='develop', ref='annotator'), text='develop', start=0, end=11)])]

In [ ]:
auxiliary_extractor.

In [49]:
auxiliary_extractor = SolidExtractor(sum(term_indeces.values(), Index()))

In [54]:
text = 'form'
sentence = Sentence(text, 0, 'testing')
auxiliary_extractor.findall(sentence)

[]

In [34]:

for pmid, goid, start, end, text in report.fn:
    for statement in godata[goid].statements:
        if len(statement.terms()) == 1 and '...' not in text:
            print(goid, text)

GO:0005694 Chromosomal
GO:0032502 form
GO:0009056 degraded
GO:0032502 establish
GO:0009058 constructed
GO:0009058 constructed
GO:0000003 Generation
GO:0065007 guided
GO:0032502 forms
GO:0009058 create
GO:0009058 create
GO:0009058 generated
GO:0009058 generated
GO:0065007 guiding
GO:0032502 establishment
GO:0032502 forming
GO:0005623 Cell
GO:0000380 Alternative splicing
GO:0032502 form
GO:0032502 formation
GO:0065007 directed
GO:0009058 anabolic activity
GO:0009058 anabolic activity
GO:0032502 formed
GO:0032502 assembly
GO:0019814 Antibody
GO:0007165 signaling pathways
GO:0000003 Generation
GO:0009058 produces
GO:0009058 produces
GO:0032502 formation
GO:0032502 formation
GO:0007601 Vision
GO:0050879 Motor
GO:0009058 yield
GO:0009058 yield
GO:0032502 formation
GO:0005623 Cells
GO:0032502 formation
GO:0032502 formation
GO:0032502 formation
GO:0009058 constructed
GO:0009058 constructed
GO:0000003 Development
GO:0009058 produced
GO:0009058 produced
GO:0065007 directing
GO:0005623 Cell
GO:00

In [20]:
auxiliary_extractor.(sentence)

[Evidence(term=Entity(lemma='neurological control of breathing', ref='GO:0002087'), text='control', start=0, end=7)]

In [21]:
finder.findall(sentence)

[Candidate<GO:0002087%000 evidences=[Evidence(term=Entity(lemma='neurological control of breathing', ref='GO:0002087'), text='control', start=0, end=7)] sentence=Sentence(text='control', offset=0, docid='diagnosing')>]

In [25]:
finder.findall(sentence)

[Candidate<GO:0002087%000 evidences=[Evidence(term=Entity(lemma='neurological control of breathing', ref='GO:0002087'), text='control', start=0, end=7)] sentence=Sentence(text='control', offset=0, docid='diagnosing')>]